## Shor's factoring algorithm

This exercise is based on the [ProjectQ](http://projectq.ch/) example. You can get the original uncommented code at [ProjectQ repository](https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/shor.py)

This algorithm is based on the paper [Parker S. and Planio M.B,Efficient factorization with a single pure qubit and logN mixed qubits](https://arxiv.org/abs/quant-ph/0001066v3)

Shor's quantum algorithm is a semiclassical algorithm where one of the steps, looking for the period od a function, is executed on a QPU because it has a better scalability with the number of bits of the number to factor. In the future, when the Quantum Computers can execute large programs, it can beat the most used asymmetric cryptographic algorithm: RSA. This algorithm is based on the assumption that it is very hard to compute the factors of a large number. Shor's algorithm and the experimental proof of the possibility of implementing it in QPUs (for factoring only very small numbers: [15](https://arxiv.org/abs/quant-ph/0112176) and [21](https://arxiv.org/abs/1111.4147)) started the Post-Quantum era, where new cryptographic algorithms are needed. 

There are several versions of the algorithm, with different requirements in the number of qubits. 

First, load the Python modules you will need to execute the code. There are some libraries that will compute the classical part, as **gcd** which calculates the Greatest Commun Divisor of two integers.

This case use [mathematical quantum maths libraries](http://projectq.readthedocs.io/en/latest/projectq.libs.math.html) and a lot of rules to decompose the [quamtum program in basic gates ](http://projectq.readthedocs.io/en/latest/projectq.setups.decompositions.html)

In [1]:
from __future__ import print_function

import math
import random
import sys
from fractions import Fraction
try:
    from math import gcd
except ImportError:
    from fractions import gcd

from builtins import input

import projectq.libs.math
import projectq.setups.decompositions
from projectq.backends import Simulator, ResourceCounter
from projectq.cengines import (MainEngine,
                               AutoReplacer,
                               LocalOptimizer,
                               TagRemover,
                               InstructionFilter,
                               DecompositionRuleSet)
from projectq.libs.math import (AddConstant,
                                AddConstantModN,
                                MultiplyByConstantModN)
from projectq.meta import Control
from projectq.ops import (X,
                          Measure,
                          H,
                          R,
                          QFT,
                          Swap,
                          get_inverse,
                          BasicMathGate,
                          All)
# Filter function, which defines the gate set for the first optimization
# (don't decompose QFTs and iQFTs to make cancellation easier)
def high_level_gates(eng, cmd):
    g = cmd.gate
    if g == QFT or get_inverse(g) == QFT or g == Swap:
        return True
    if isinstance(g, BasicMathGate):
        return False
        if isinstance(g, AddConstant):
            return True
        elif isinstance(g, AddConstantModN):
            return True
        return False
    return eng.next_engine.is_available(cmd)

Create the Engine. This time, several optimizations will be included. The program will also count the needed resources using the Engine ResourceCounter.

In [2]:
# build compilation engine list
resource_counter = ResourceCounter()
rule_set = DecompositionRuleSet(modules=[projectq.libs.math,
                                         projectq.setups.decompositions])
compilerengines = [AutoReplacer(rule_set),
                   InstructionFilter(high_level_gates),
                   TagRemover(),
                   LocalOptimizer(3),
                   AutoReplacer(rule_set),
                   TagRemover(),
                   LocalOptimizer(3),
                   resource_counter]

# make the compiler and run the circuit on the simulator backend
eng = MainEngine(Simulator(), compilerengines)

Choose the integer to factor. Execute the next cell and answer the question. Select a small number as 15 or 21 and odd, or you can wait a long time! or the solution is trivial.

In [3]:
# print welcome message and ask the user for the number to factor
N=2
while gcd(2,N)==2:
    print("\n\t\033[37mprojectq\033[0m\n\t--------\n\tImplementation of Shor"
      "\'s algorithm.", end="")
    N = int(input('\n\tNumber to factor: '))
    if gcd(2,N)==2:
        print("\n\n%d is EVEN. Please, select another integer"%N)
    
print("\n\tFactoring N = {}: \033[0m".format(N), end="")


	projectq
	--------
	Implementation of Shor's algorithm.
	Number to factor: 21

	Factoring N = 21: 

Ramdomly, the algorithm selects one initial factor and tests if it is a factor.

In [4]:
# choose a base at random:
a = int(random.random()*N)
if not gcd(a, N) == 1:
    print("\n\n\t\033[92mOoops, we were lucky: Chose non relative prime"
          " by accident :)")
    print("\tFactor: {}\033[0m".format(gcd(a, N)))

print("\tInitial guess: {}\033[0m".format(a))


	Initial guess: 17


Now, this is the Quantum part: find the period of a function. The origial idea is due to Simon who sent it to a conference where Shor was a reviewer. Lucky person, because he knew that finding a period of a function was the weak point of the factorisation. And now, he could do it!.

First of all, we need to calculate the number of qubits **n** that are needed. Afterwards, we allocate a Quantum register with this number of qubits. This register is initialized to $|0\rangle^{\otimes n-1} \otimes |1\rangle$

In [5]:
n = int(math.ceil(math.log(N, 2)))

x = eng.allocate_qureg(n)

X | x[0]

measurements = [0] * (2 * n)  # will hold the 2n measurement results

print("Algorithm will use %d qubits to produce %d measurements"%(n+1,2*n))


Algorithm will use 6 qubits to produce 10 measurements


We have to allocate one additional qubit. This is the only one that have to be measured!.

In [6]:
ctrl_qubit = eng.allocate_qubit()

Ok. Now this is the trick: The quantum algorithm. Because ProjectQ has some quantum routines implemented as Multiplication, it is really easy to program it. This loop implements this circuit:

<img src="Images/short.jpg"/>

Where each $R'_j(\phi'_j)$ is a phase shift gate with an angle $$\phi'_j={-2\pi \sum_{k=2}^{j}\frac{m_{j-k}}{2^k}}$$

In [7]:
for k in range(2 * n):
    current_a = pow(a, 1 << (2 * n - 1 - k), N)

    H | ctrl_qubit
    with Control(eng, ctrl_qubit):
        MultiplyByConstantModN(current_a, N) | x

    for i in range(k):
        if measurements[i]:
            R(-math.pi/(1 << (k - i))) | ctrl_qubit
    H | ctrl_qubit

    # and measure
    Measure | ctrl_qubit
    eng.flush()
    measurements[k] = int(ctrl_qubit)
    if measurements[k]:
        X | ctrl_qubit

    
    print("\033[95m{}\033[0m".format(measurements[k]), end="")
    sys.stdout.flush()

All(Measure) | x

0000000000

Now, we conver the results to the final period, using classical methods.

In [8]:
# turn the measured values into a number in [0,1)
y = sum([(measurements[2 * n - 1 - i]*1. / (1 << (i + 1)))
         for i in range(2 * n)])

# continued fraction expansion to get denominator (the period?)
r = Fraction(y).limit_denominator(N-1).denominator


And we try to find the factors. Becareful, the algorithm can fail to find a factor.

In [9]:
# try to determine the factors
if r % 2 != 0:
    r *= 2
apowrhalf = pow(a, r >> 1, N)
f1 = gcd(apowrhalf + 1, N)
f2 = gcd(apowrhalf - 1, N)
if ((not f1 * f2 == N) and f1 * f2 > 1 and
        int(1. * N / (f1 * f2)) * f1 * f2 == N):
    f1, f2 = f1*f2, int(N/(f1*f2))
if f1 * f2 == N and f1 > 1 and f2 > 1:
    print("\n\n\t\033[92mFactors found :-) : {} * {} = {}\033[0m"
          .format(f1, f2, N))
else:
    print("\n\n\t\033[91mBad luck: Found {} and {}\033[0m".format(f1,
                                                                  f2))




	Factors found :-) : 3 * 7 = 21


And we print the final resources we have used.

In [10]:
print(resource_counter)  # print resource usage

Gate class counts:
    AllocateQubitGate : 166
    CCR : 1476
    CR : 7176
    CSwapGate : 50
    CXGate : 200
    DeallocateQubitGate : 160
    HGate : 2592
    MeasureGate : 15
    R : 600
    XGate : 201

Gate counts:
    Allocate : 166
    CCR(0.098174770425) : 20
    CCR(0.196349540849) : 32
    CCR(0.392699081699) : 72
    CCR(0.490873852124) : 20
    CCR(0.785398163397) : 80
    CCR(0.981747704246) : 40
    CCR(1.079922474671) : 20
    CCR(1.178097245096) : 14
    CCR(1.276272015521) : 12
    CCR(1.570796326795) : 186
    CCR(1.66897109722) : 20
    CCR(1.865320638069) : 8
    CCR(1.963495408494) : 70
    CCR(2.159844949343) : 20
    CCR(2.356194490192) : 64
    CCR(2.552544031041) : 16
    CCR(2.94524311274) : 10
    CCR(3.14159265359) : 300
    CCR(3.337942194439) : 20
    CCR(3.730641276138) : 14
    CCR(3.926990816987) : 86
    CCR(4.123340357836) : 10
    CCR(4.319689898686) : 50
    CCR(4.417864669111) : 4
    CCR(4.61421420996) : 10
    CCR(4.712388980385) : 114
    CCR(